# Writing data to disk

Once we've loaded data we may want to save it to some on-disk format. Satpy provides multiple formats that it can save our data to using "writer" objects. Let's use Satpy's utility functions to learn what readers are available to us:

In [ ]:
%run ../init_notebook.py
from satpy import available_writers
sorted(available_writers())

In the above output you should see 3-5 writers depending on how your python environment is set up. In this tutorial we'll be using the `geotiff`, `simple_image`, and `cf` writers. If we are missing any of the dependencies for some of Satpy's writers they won't be listed here. We can check what functionality we are missing by using `check_satpy` like we did in the [Introduction](./01_introduction.ipynb).

## Images - GeoTIFF

We'll start out learning how to save geotiffs, Satpy's default writer, using the same data we did in the previous lesson. Let's create a `Scene` object and pick one of the 16 ABI channels (ex. `C07`).

In [ ]:
from satpy import Scene
from glob import glob

my_channel = 'EDITME'
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.load([my_channel])

One of the simplest ways to save all loaded data to disk is to use the `save_datasets` method of the `Scene`.

In [ ]:
scn.save_datasets()

By default this method will save files to the current working directory (`.`).

Using notebook's `!` functionality we can run command line tools and investigate the file using the GDAL project's [gdalinfo](https://gdal.org/programs/gdalinfo.html):

In [ ]:
!pwd
!ls

In [ ]:
!gdalinfo C07_20180511_213220.tif

Satpy has created an LA, or Luminance + Alpha, striped geotiff image compressed with the "DEFLATE" compression algorithm. From the `gdalinfo` output we can see information like the map projection parameters formatted in Well-known Text (WKT), image size, and image data type. GeoTIFF is not a standard web image format so we can't display them out-of-the-box in our browser. They are typically used by GIS tools like ArcGIS, QGIS, or with Web Mapping Services (WMS). Most image viewers can also display these images so navigating to the image file and opening it (double click) should open the image with ImageMagick on linux, Preview on OSX, and Windows Photo Viewer on Windows.

Under the hood Satpy is using the `rasterio` and `GDAL` libraries to create the GeoTIFF. GDAL has [a lot of options](https://gdal.org/drivers/raster/gtiff.html#creation-options) for GeoTIFF creation and Satpy allows us to specify these when saving. Let's create a tiled GeoTIFF image which can be useful when hosting images on cloud services (see [COG](https://www.cogeo.org/) for details) and typically produces smaller files. We'll also use a dask utility called `ProgressBar` to provide feedback on the calculation.

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    scn.save_datasets(tiled=True, copy_src_overviews=True)

Satpy overwrote the file we generated before. Let's use `gdalinfo` again to investigate the file.

In [ ]:
!ls

In [ ]:
!gdalinfo C07_20180511_213220.tif

The main difference is the `Block=256x256` which is telling us that our data was stored in 256x256 chunks; the recommended way of storing data for a Cloud Optimized Geotiff.

We can see in the `gdalinfo` output above that our image data is of type `Byte` meaning it is an 8-bit unsigned integer. Sometimes you may want to store data in a GeoTIFF as floating point numbers. Let's do one more GeoTIFF example and use some additional keyword arguments to customize our output.

In [ ]:
import numpy as np

with ProgressBar():
    scn.save_datasets(base_dir='float_images', filename='{name}.float.tif',
                      dtype=np.float32, enhance=False)

In [ ]:
!ls

In [ ]:
!gdalinfo float_images/C07.float.tif

We've now created a 32-bit float GeoTIFF to use in GIS applications or other software to do more analysis. We used the `base_dir` keyword argument to specify a new directory where we wanted our files saved. We specified the `enhance=False` argument to tell Satpy to not scale observed data values (Kelvin, percentages, etc) to image values (0-255); there will be more on this in later lessons. We also provided a custom filenaming scheme with the `filename` keyword argument.

### Exercise

**Time: 5 minutes**

A key point here is that the filenaming scheme can use any attribute that is in a DataArray's `.attrs` dictionary. To try this, update the filename pattern in the previous `save_datasets` call to add the `standard_name` or `start_time` or any other attribute. What filenames are produced?

## Images - PNG, JPEG, etc

So far we've created GeoTIFFs using the `'geotiff'` writer, but we can also create PNGs and other basic image formats by using the `'simple_image'` writer which takes advantage of the many formats supported by the PIL package.

In [ ]:
with ProgressBar():
    scn.save_datasets(writer='simple_image')

In [ ]:
!ls

We could view this image like we did the GeoTIFF, with our system's image viewer, or we could view it in the browser. However, by default Satpy has saved the entire image array to the file so viewing a couple of them may be an issue for our browser. In future lessons we'll learn of ways to change the resolution of our data before saving it to disk.

Instead of specifying the `writer` explicitly we could have let Satpy determine the best writer to use based on the filename. Using what we learned about `filename` before, let's update the below call to save a PNG image with the `.png` extension.

In [ ]:
scn.save_datasets(editme=editme)

# CF-compliant NetCDF Files

In addition to image formats, we may want to save our data in more common data file formats. A very common format when working with satellite imagery data is the NetCDF format (.nc) with metadata matching the Climate and Forecast (CF) metadata conventions. We'll use the `'cf'` writer for this, the default writer for any file ending in `.nc`.

First, let's load a couple extra channels from our Scene.

In [ ]:
scn.load(['C08', 'C09'])

In [ ]:
with ProgressBar():
    scn.save_datasets(filename='my_abi_data.nc')

In [ ]:
!ls

In [ ]:
!ncdump -h my_abi_data.nc

Note how this handled the x/y coordinate variables for us as well as the CF `grid_mapping` variable. By default Satpy also created `longitude` and `latitude` variables for users who are more comfortable with degrees instead of projection meters. There are [a lot of options](https://satpy.readthedocs.io/en/latest/api/satpy.writers.html#module-satpy.writers.cf_writer) we can configure for this writer including to turn of the longitude and latitude variable creation, grouping variables, and more. Exploring these options is out of the scope for this tutorial and is left as an exercise for the reader.

### Further Research

 * [Scene.save_dataset][1]
 * [GeoTIFF creation options][2]
 * [NetCDF creation options][3]

  [1]: https://satpy.readthedocs.io/en/latest/api/satpy.html#satpy.scene.Scene.save_dataset
  [2]: https://gdal.org/drivers/raster/gtiff.html#creation-options
  [3]: https://satpy.readthedocs.io/en/latest/api/satpy.writers.html#module-satpy.writers.cf_writer